Chest CT Data generation with a WGAN

In [1]:
import kagglehub

# https://www.kaggle.com/datasets/mohamedhanyyy/chest-ctscan-images
path = kagglehub.dataset_download("mohamedhanyyy/chest-ctscan-images")

print("Path to dataset files:", path)

100%|██████████| 119M/119M [00:17<00:00, 7.08MB/s] 

Extracting files...


Path to dataset files: C:\Users\ppere\.cache\kagglehub\datasets\mohamedhanyyy\chest-ctscan-images\versions\1


In [14]:
!mv .cache\kagglehub\datasets\mohamedhanyyy\chest-ctscan-images\versions\1 .

"mv" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.


El modificador no es v lido: "..".


In [18]:
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torch.utils.data as data_utils
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torch import autograd
import time as t
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import os
from itertools import chain
from torchvision import utils
import tensorflow as tf


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.9_3.9.3568.0_x64__qbz5n2kfra8p0\lib\runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.9_3.9.3568.0_x64__qbz5n2kfra8p0\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "C:\Users\ppere\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\ipykernel_launche

AttributeError: _ARRAY_API not found

ImportError: numpy.core.multiarray failed to import

# Dataloader

In [19]:
# path= "chest-ctscan-images/versions/1/Data/"
path = "../../../ChestCTKaggle/Data"

def get_data_loader(args):
      train_path = path+"train"
      valid_path = path+"valid"
      test_path = path+"test"

      transform = transforms.Compose([
        transforms.Grayscale(),
        transforms.Resize((64,64)),
        transforms.ToTensor(),  # Convertir a tensor
        transforms.Normalize((0.5,), (0.5,))  # Normalización a [-1, 1]
      ])
        # Cargar los datasets
      train_dataset = datasets.ImageFolder(root=train_path, transform=transform)
      valid_dataset = datasets.ImageFolder(root=valid_path, transform=transform)
      test_dataset = datasets.ImageFolder(root=test_path, transform=transform)

      # Verificar el tamaño del dataset
      print(f"===> Tamaño del conjunto de entrenamiento: {len(train_dataset)}")
      print(f"===> Tamaño del conjunto de validación: {len(valid_dataset)}")
      print(f"===> Tamaño del conjunto de prueba: {len(test_dataset)}")

      train_dataloader = data_utils.DataLoader(train_dataset, batch_size=args.batch_size, shuffle=True)
      test_dataloader = data_utils.DataLoader(test_dataset,  batch_size=args.batch_size, shuffle=True)

      return train_dataloader, test_dataloader


# Generator and Discriminator

In [20]:
import WGAN as wgan


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.9_3.9.3568.0_x64__qbz5n2kfra8p0\lib\runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.9_3.9.3568.0_x64__qbz5n2kfra8p0\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "C:\Users\ppere\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\ipykernel_launche

AttributeError: _ARRAY_API not found

ImportError: numpy.core.multiarray failed to import

# Exec

In [21]:
import argparse
import os

"""
python main.py --model WGAN-GP --is_train True --download True --dataset chestCT --generator_iters 40000 --batch_size 64
"""

def parse_args():
    parser = argparse.ArgumentParser(description="Pytorch implementation of GAN models.")

    parser.add_argument('--model', type=str, default='WGAN-GP', choices=['GAN', 'DCGAN', 'WGAN-CP', 'WGAN-GP'])
    parser.add_argument('--is_train', type=str, default='True')
    parser.add_argument('--dataroot', required=False, help='path to dataset')
    parser.add_argument('--dataset', type=str, default='mnist', choices=['mnist', 'fashion-mnist', 'chestCT'],
                            help='The name of dataset')
    parser.add_argument('--download', type=str, default='False')
    parser.add_argument('--epochs', type=int, default=50, help='The number of epochs to run')
    parser.add_argument('--batch_size', type=int, default=64, help='The size of batch')
    parser.add_argument('--cuda',  type=str, default='False', help='Availability of cuda')

    parser.add_argument('--load_D', type=str, default='False', help='Path for loading Discriminator network')
    parser.add_argument('--load_G', type=str, default='False', help='Path for loading Generator network')
    parser.add_argument('--generator_iters', type=int, default=10000, help='The number of iterations for generator in WGAN model.')
    return check_args(parser.parse_args())


# Checking arguments
def check_args(args):
    # --epoch
    try:
        assert args.epochs >= 1
    except:
        print('Number of epochs must be larger than or equal to one')

    # --batch_size
    try:
        assert args.batch_size >= 1
    except:
        print('Batch size must be larger than or equal to one')

    if args.dataset == 'cifar' or args.dataset == 'stl10':
        args.channels = 3
    else:
        args.channels = 1
    args.cuda = True if args.cuda == 'True' else False
    return args




In [22]:
# parse_args
# get_data_loader

def main(args):
    model = None
    if args.model == 'GAN':
        print("Modelo GAN no creado")
    elif args.model == 'DCGAN':
        print("Modelo DCGAN no creado")
    elif args.model == 'WGAN-CP':
        print("Modelo WGAN-CP no creado")
    elif args.model == 'WGAN-GP':
        print("Has elegido WGAN-GP")
        model = wgan.WGAN_GP(args)
    else:
        print("Model type non-existing. Try again.")
        exit(-1)

    # Load datasets to train and test loaders
    train_loader, test_loader = get_data_loader(args)
    #feature_extraction = FeatureExtractionTest(train_loader, test_loader, args.cuda, args.batch_size)

    # Start model training
    if args.is_train == 'True':
        model.train(train_loader)

    # start evaluating on test data
    else:
        model.evaluate(test_loader, args.load_D, args.load_G)
        for i in range(50):
           model.generate_latent_walk(i)


In [23]:
class Args:
    def __init__(self):
        self.model = 'WGAN-GP'  # Cambia según el modelo que deseas usar
        self.is_train = 'True'   # Cambia a 'False' si solo quieres evaluar
        self.batch_size = 32      # Ajusta según tu configuración
        self.cuda = True           # Cambia según si usas CUDA
        self.load_D = None        # Ruta para cargar el modelo del discriminador
        self.load_G = None
        self.channels = 1            # Número de canales (por ejemplo, 3 para imágenes RGB)
        self.generator_iters = 10

args = Args()

main(args)

Has elegido WGAN-GP


NameError: name 'wgan' is not defined